<a href="https://colab.research.google.com/github/NIMBL-UBC/MediaPipe/blob/main/MediaPipe_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing MediaPipe
!pip install -q mediapipe==0.10.0

In [ ]:
#Import the required libraries

import cv2
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import mediapipe as mp
#from mediapipe.tasks import python
#from mediapipe.tasks.python import vision
from google.colab.patches import cv2_imshow
from tabulate import tabulate
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from shutil import rmtree

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
#Link your google drive where the files to be evaluated are stored

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_frames_count(video_path, folder_path):
    # Check if the folder exists, if not create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Function to extract frames
    def FrameCapture(path):
        # Path to video file
        vidObj = cv2.VideoCapture(path)
        # Used as a counter variable
        count = 0
        # Checks whether frames were extracted
        success = 1
        while success:
            # Frame is extracted
            success, image = vidObj.read()
            if not success:
                break
            # Frame is stored in the assigned folder with a specific number
            cv2.imwrite(os.path.join(folder_path, "frame%d.jpg" % count), image)
            count += 1
        return count  # Return the value of 'count' when the function finishes

    # Call the FrameCapture function to extract frames and get the value of 'count'
    countf = FrameCapture(video_path)

    # To check if the code can read files
    testimg = "frame1.jpg"
    image_path = os.path.join(folder_path, testimg)

    # Check if the file exists
    if not os.path.exists(image_path):
        print("Image file does not exist")
    else:
        # Read the image using cv2.imread()
        image = cv2.imread(image_path)
        if image is not None:
            print('The image can be read by the code')
        else:
            # Unable to read the image
            print("Error reading the image")

    # Check if the image was read successfully
    if image is not None:
        print("The image exists, proceed with code")
    else:
        # Unable to read the image
        print("Error reading the image")

    # Return the value of 'count' as the output of the function
    return countf

In [ ]:
def extract_coords(folder_path, count):
    x = []
    y = []

    for i in range(count):
        filename = f"/frame{i}.jpg"
        frameloc = folder_path + filename # Path of folder where frames have been extracted
        #print(frameloc)
        image = cv2.imread(frameloc)
        #if image is not None:
          #print("yes queen")
        with mp_hands.Hands(
              static_image_mode=True,
              max_num_hands=1, #{change between 1 or 2 ONLY}
              min_detection_confidence=0.5) as hands:
              #print(image.shape)
              results = hands.process(image)
              #print(type(image))
              results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
              # Print handedness and draw hand landmarks on the image.
              #print('Handedness:', results.multi_handedness) #{uncomment this line to print left or right handedness}
              image_height, image_width, _ = image.shape
              annotated_image = image.copy()
              if results is not None:
                if results.multi_hand_landmarks is not None:
                  for hand_landmarks in results.multi_hand_landmarks:
                  #print('hand_landmarks:', hand_landmarks) #{uncomment this line to print ALL landmarks for a single frame}
                    print("."
                        #f'Index finger tip coordinates: (',
                        #f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                        #f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                    )
                  x.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width)
                  y.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height)
              else:
                #print("No hand landmarks found.")
                continue
    shutil.rmtree(folder_path)
    return x, y

In [ ]:
def std_data(x, y, vn):
    table = list(zip(x, y))
    df = pd.DataFrame(table, columns=[f'x{vn}', f'y{vn}'])
    existing_rows = len(df[f'x{vn}'])
    zero_rows = 1500 - existing_rows
    additional_zeros = pd.DataFrame({f'x{vn}': np.zeros(zero_rows),
                                     f'y{vn}': np.zeros(zero_rows)})
    df = pd.concat([df, additional_zeros], ignore_index=True)

    time = np.linspace(1, len(x), num=len(x))
    nintp = 1500 - len(x)
    newx = []
    splinex = InterpolatedUnivariateSpline(time, x, k=3)
    newx = splinex(np.linspace(time.min(), time.max(), nintp))

    newy = []
    spliney = InterpolatedUnivariateSpline(time, y, k=3)
    newy = spliney(np.linspace(time.min(), time.max(), nintp))

    joined_x = np.concatenate((x, newx))
    joined_y = np.concatenate((y, newy))

    df[f'resx{vn}'] = joined_x
    df[f'resy{vn}'] = joined_y

    #normx = scale(joined_x, with_mean=True, with_std=True)
    #normy = scale(joined_y, with_mean=True, with_std=True)

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    joined_x_reshaped = joined_x.reshape(-1, 1)
    joined_y_reshaped = joined_y.reshape(-1, 1)

    normx = scaler_x.fit_transform(joined_x_reshaped)
    normy = scaler_y.fit_transform(joined_y_reshaped)

    df[f'normx{vn}'] = normx
    df[f'normy{vn}'] = normy

    return df

In [ ]:
#Path to video location, and the location of the folder where you want the frames to be extracted
vpath = '/content/drive/MyDrive/videos/'
fpath = '/content/drive/MyDrive/output_frames_'
ext = ".mp4"
df_final = pd.DataFrame()
df = pd.DataFrame()

#trials = ["1691","1692","1693","1694","1695","1696","1697","1698","1699","16910","1711","1712","1713","1714","1715","1716","1717","1718","1719","17110","1731","1732","1733","1734","1735","1736","1737","1738","1739","17310"]
trials = ["1281","1282","1283","1284","1285","1286","1287","1288","1289","12810"]
#"1121","1122","1123","1124","1125","1126","1127","1128","1129","11210","1251","1252","1253","1254","1255","1256","1257","1258","1259","12510","1321","1322","1323","1324","1325","1326","1327","1328","1329","13210","1341","1342","1343","1344","1345","1346","1347","1348","1349","13410","1381","1382","1383","1384","1385","1386","1387","1388","1389","13810","1401","1402","1403","1404","1405","1406","1407","1408","1409","14010","1411","1412","1413","1414","1415","1416","1417","1418","1419","14110","1431","1432","1433","1434","1435","1436","1437","1438","1439","14310","1691","1692","1693","1694","1695","1696","1697","1698","1699","16910","1711","1712","1713","1714","1715","1716","1717","1718","1719","17110","1731","1732","1733","1734","1735","1736","1737","1738","1739","17310"

for trial in trials:
  video_path = os.path.join(vpath + trial + ext)
  folder_path = os.path.join(fpath + trial)
  vn = trial
  countt=get_frames_count(video_path, folder_path)
  x,y = extract_coords(folder_path,countt)
  df = std_data(x,y,vn)
  df_final = pd.concat([df_final, df], axis =1)

print(df_final)

In [ ]:
df_final.to_excel('MediaPipeData_128.xlsx', index=False)

In [ ]:
df_final

In [ ]:
df1 = pd.read_excel('MediaPipeData_1251to13810series.xlsx')
df2 = pd.read_excel('MediaPipeData_1401to1435series.xlsx')
df3 = pd.read_excel('MediaPipeData_1691to17310series.xlsx')
df_expo= pd.concat([df1, df2, df3], axis =1)
df_expo
df_expo.to_excel('MediapipeJPdata.xlsx', index=False)

In [ ]:
import pandas as pd
path = r"/content/drive/MyDrive"
dfTC_final = pd.DataFrame()

def tlc_extract(id, s, t, vt):
    tracelabdata = pd.read_csv(path + "/tracings_2023VWupdate.txt", sep="\t")
    temp = tracelabdata[tracelabdata['id'] == id]
    plotDat = temp[temp['session'] == s]
    trialcheck = plotDat[plotDat['trial'] == t]
    dfTC = trialcheck[['x', 'y','trial']]
    dfTC = std_data(dfTC['x'],dfTC['y'], vt)
    #column_names = [f'tlcx{vt}', f'tlcy{vt}', f'trial{vt}']
    #dfTC.columns = column_names
    dfTC.reset_index(drop=True, inplace=True)
    return dfTC

In [ ]:
dfTC = pd.DataFrame()
dfTC_final = pd.DataFrame()
dfTC_finalest = pd.DataFrame()

In [ ]:
id = [26, 28, 31, 32, 33, 33, 32, 35, 36, 37, 38, 39]
s = [3, 4, 3, 1, 1, 4, 4, 1, 1, 1, 1, 1]
t = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

vt = ["1121","1122","1123","1124","1125","1126","1127","1128","1129","11210","1251","1252","1253","1254","1255","1256","1257","1258","1259","12510","1281","1282","1283","1284","1285","1286","1287","1288","1289","12810","1321","1322","1323","1324","1325","1326","1327","1328","1329","13210","1341","1342","1343","1344","1345","1346","1347","1348","1349","13410","1381","1382","1383","1384","1385","1386","1387","1388","1389","13810","1401","1402","1403","1404","1405","1406","1407","1408","1409","14010","1411","1412","1413","1414","1415","1416","1417","1418","1419","14110","1431","1432","1433","1434","1435","1691","1692","1693","1694","1695","1696","1697","1698","1699","16910","1711","1712","1713","1714","1715","1716","1717","1718","1719","17110","1731","1732","1733","1734","1735","1736","1737","1738","1739","17310"]
import numpy as np
j = 9
#n = 0
while j < 12:
  print(j)
  for i in t:
    print(id[j],s[j],i,vt[n])
    dfTC = tlc_extract(id[j],s[j],i,vt[n])
    dfTC_final = pd.concat([dfTC_final, dfTC], axis =1)
    n = n+1
  j=j+1

print(dfTC_final)

In [ ]:
dfTC_final.to_excel('TracelabDataforJP_final.xlsx', index=False)

In [ ]:
tlcx = dfTC[f'tlcx{vt}']
tlcy = dfTC[f'tlcy{vt}']

tlctime = np.linspace(1,len(dfTC[f'tlcx{vt}']),num=len(dfTC[f'tlcx{vt}']))

tlcnintp = 250 - len(dfTC[f'tlcx{vt}'])  # Number of points for interpolation

# Perform spline interpolation
tlcnewx = []
tlcsplinex = InterpolatedUnivariateSpline(tlctime, tlcx, k=3)
tlcnewx = tlcsplinex(np.linspace(tlctime.min(), tlctime.max(), tlcnintp))

tlcnewy=[]
tlcspliney = InterpolatedUnivariateSpline(tlctime, tlcy, k=3)
tlcnewy = tlcspliney(np.linspace(tlctime.min(), tlctime.max(), tlcnintp))

print(tlcnewx)
print(len(tlcnewx))

In [ ]:
tlcexisting_rows = len(dfTC[f'tlcx{vt}'])

tlczero_rows = 250 - tlcexisting_rows

# Create a DataFrame with the additional rows of zeros
tlcadditional_zeros = pd.DataFrame({f'tlcx{vt}': np.zeros(tlczero_rows),
                                 f'tlcx{vt}': np.zeros(tlczero_rows)})

# Concatenate the existing DataFrame with the additional zeros DataFrame
dfTC = pd.concat([dfTC, tlcadditional_zeros], ignore_index=True)


In [ ]:
joined_tlcx = np.concatenate((tlcx, tlcnewx))
joined_tlcy = np.concatenate((tlcy, tlcnewy))

dfTC[f'tlcresx{vt}'] = joined_tlcx
dfTC[f'tlcresy{vt}'] = joined_tlcy

from sklearn.preprocessing import scale

tlcnormx = scale(joined_tlcx, with_mean=True, with_std=True)
tlcnormy = scale(joined_tlcy, with_mean=True, with_std=True)

dfTC[f'tlcnormx{vt}'] = tlcnormx
dfTC[f'tlcnormy{vt}'] = tlcnormy

plt.scatter(tlcnormx, tlcnormy)
len(tlcnormx)
print(len(joined_tlcy))
print(dfTC)


In [ ]:
df_TLC = pd.concat([df_TLC, dfTC],axis = 1)
print(df_TLC)

In [ ]:
df_TLC.to_excel('TraceLabData.xlsx', index=False)

In [ ]:
# To run procrutes analysis using a python library - not
import numpy as np
from scipy.spatial import procrustes
import matplotlib.pyplot as plt

# Assuming you have the 'MP' and 'TC' arrays defined in Python

# Perform procrustes analysis
mtx1, mtx2, disparity = procrustes(dfMP, dfTC)
disparity

# # Retrieve the results
# PA = result[0]
# disparity = result[1]

# # Print summary
# print("Translation:", PA['translation'])
# print("Rotation:", PA['rotation'])
# print("Scale:", PA['scale'])
# print("Disparity:", disparity)

# # Plot procrustes analysis results
# plt.scatter(PA['source'][:, 0], PA['source'][:, 1], label='MP')
# plt.scatter(PA['target'][:, 0], PA['target'][:, 1], label='TC')
# plt.legend()
# plt.show()

In [ ]:
# # Generate 250 points for interpolation
# interp_func = interp1d(np.arange(len(x)), x, kind='cubic')
# #interp_funcy = interp1d(time, y, kind='cubic')
# x_interp = interp_func(np.linspace(0, len(x) - 1, 250 * len(x)))
# #y_new = interp_funcy(time)
# x_new

# from scipy.interpolate import BSpline, make_interp_spline
# b = make_interp_spline(time, x)